# EDA

In [38]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [39]:
df = pd.read_csv("data/personal_transactions.csv")
df.Date = pd.to_datetime(df.Date)

In [40]:
df.head()

,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card


In [59]:
df.shape

(806, 6)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              806 non-null    datetime64[ns]
 1   Description       806 non-null    object        
 2   Amount            806 non-null    float64       
 3   Transaction Type  806 non-null    object        
 4   Category          806 non-null    object        
 5   Account Name      806 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 37.9+ KB


In [42]:
debits = df[df["Transaction Type"] == 'debit']

In [43]:
def count_sum(data ,column: str, plot = "Pie"):
    by_column = data\
                .groupby(column)\
                .agg({"Transaction Type": "count", "Amount": "sum"})\
                .rename(columns={"Transaction Type": "Total"})\
                .reset_index()
    by_column.columns = [column, "Total", "Sum"]
    
    labels = by_column[column]

    fig = None
    
    if plot == 'Pie':
        fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Total"],
                name="Total"
            ),
        1, 2)
    elif plot == 'Scatter':
        fig = make_subplots(1, 2,
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Total"],
                name="Total"
            ),
        1, 2)
        

    fig.update_layout(title_text=f"{column} Analysis")
    fig.show()
    
    return by_column

In [44]:
# gastos_by_cat = count_sum(debits, "Category")
# gastos_by_cat

In [56]:
df["Description"].value_counts()

Credit Card Payment    143
Grocery Store          103
Amazon                  59
Biweekly Paycheck       46
Hardware Store          34
                      ... 
Irish Pub                1
Tiny Deli                1
Chevron                  1
Japanese Restaurant      1
Sheetz                   1
Name: Description, Length: 65, dtype: int64